In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np

import datetime as dt

import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

import sqlite3

from scipy.stats.mstats import winsorize

from joblib import load

<IPython.core.display.Javascript object>

In [3]:
def ilinechart(df, x, y, groups=None, title=""):
    fig = px.line(df, x=x, y=y, color=groups, title=title, template="none").update(
        layout=dict(title=dict(x=0.5))
    )

    fig.show()

<IPython.core.display.Javascript object>

In [4]:
# Connect to DB
conn = sqlite3.connect("btc.db")
c = conn.cursor()

<IPython.core.display.Javascript object>

In [5]:
data = pd.read_sql_query("SELECT * FROM book_imbalances_lag_10", conn)

<IPython.core.display.Javascript object>

In [6]:
data = data.drop(columns=["index"])
data["time"] = pd.to_datetime(data["time"])
data = data.set_index("time", drop=False)
data = data.tz_localize(tz="US/Eastern")

data

,time,lag period,midprice,oir,voi,tfi
time,,,,,,
2020-07-02 13:57:49.545534-04:00,2020-07-02 13:57:49.545534,10,9069.065,-4.353743,3.770439,0.651967
2020-07-02 13:58:01.812770-04:00,2020-07-02 13:58:01.812770,10,9070.005,-1.143382,-11.737757,0.672478
2020-07-02 13:58:13.318442-04:00,2020-07-02 13:58:13.318442,10,9069.815,-0.103770,1.288612,0.376768
2020-07-02 13:58:24.735725-04:00,2020-07-02 13:58:24.735725,10,9070.455,0.109324,-7.522974,0.658716
2020-07-02 13:58:36.500687-04:00,2020-07-02 13:58:36.500687,10,9068.015,0.058965,-38.674267,0.449309
...,...,...,...,...,...,...
2020-09-03 09:32:49.476146-04:00,2020-09-03 09:32:49.476146,10,10830.955,-0.043333,-22.368282,0.461289
2020-09-03 09:33:00.791797-04:00,2020-09-03 09:33:00.791797,10,10830.955,-0.489244,-21.341815,0.470896
2020-09-03 09:33:12.133621-04:00,2020-09-03 09:33:12.133621,10,10830.955,-0.921108,-22.820611,0.468831


<IPython.core.display.Javascript object>

In [7]:
# Winsorize features
# cols = ["oir", "voi", "tfi"]
# for col in cols:
#     data[col] = winsorize(data[col], (0.05, 0.05))

<IPython.core.display.Javascript object>

In [8]:
data.columns

Index(['time', 'lag period', 'midprice', 'oir', 'voi', 'tfi'], dtype='object')

<IPython.core.display.Javascript object>

In [9]:
# data = data.loc[data["time"] > "2020-07-24 04:56:29.780165"]

<IPython.core.display.Javascript object>

In [10]:
# Transform dataframe and add target column which indicates if price moves up or down
transformed = data.drop(columns=["lag period"]).reset_index(drop=True)
history = 500
future = 500

shifts = [x + future for x in list(range(1, history + 1))]
for shift in shifts:
    transformed["t-oir-" + str(shift)] = transformed["oir"].shift(shift)
    transformed["t-voi-" + str(shift)] = transformed["voi"].shift(shift)
    transformed["t-tfi-" + str(shift)] = transformed["tfi"].shift(shift)
    transformed["t-midprice-" + str(shift)] = transformed["midprice"].shift(shift)

transformed = transformed.dropna()
transformed["target"] = 0
transformed.loc[transformed["midprice"] > transformed["t-midprice-501"], "target"] = 1
transformed.set_index("time").drop(["oir", "voi", "tfi"], axis=1)

transformed

,time,midprice,oir,voi,tfi,t-oir-501,t-voi-501,t-tfi-501,t-midprice-501,t-oir-502,...,t-midprice-998,t-oir-999,t-voi-999,t-tfi-999,t-midprice-999,t-oir-1000,t-voi-1000,t-tfi-1000,t-midprice-1000,target
1000,2020-07-02 20:30:07.844349,9093.895,1.380148,-1.042959,0.461344,-0.943778,-7.203480,0.760322,9039.495,-1.171831,...,9069.815,-1.143382,-11.737757,0.672478,9070.005,-4.353743,3.770439,0.651967,9069.065,1
1001,2020-07-02 20:30:32.826776,9094.025,1.270786,-3.310276,0.447799,-3.057961,7.477914,0.722981,9038.645,-0.943778,...,9070.455,-0.103770,1.288612,0.376768,9069.815,-1.143382,-11.737757,0.672478,9070.005,1
1002,2020-07-02 20:30:45.469928,9093.885,0.550035,-1.078092,0.454933,-1.011289,-7.149358,0.673969,9038.540,-3.057961,...,9068.015,0.109324,-7.522974,0.658716,9070.455,-0.103770,1.288612,0.376768,9069.815,1
1003,2020-07-02 20:30:59.034912,9093.660,0.010264,0.605181,0.480753,-0.011996,-8.574217,0.753685,9038.175,-1.011289,...,9067.925,0.058965,-38.674267,0.449309,9068.015,0.109324,-7.522974,0.658716,9070.455,1
1004,2020-07-02 20:31:11.771165,9093.995,1.110706,10.997174,0.451397,-0.634792,-13.929421,0.631556,9039.585,-0.011996,...,9067.925,1.977738,9.186586,0.430767,9067.925,0.058965,-38.674267,0.449309,9068.015,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285304,2020-09-03 09:32:49.476146,10830.955,-0.043333,-22.368282,0.461289,0.386811,-4.001545,0.539549,11237.055,-0.260199,...,11320.340,-1.570151,-16.401193,1.429919,11320.495,-1.441880,-15.786901,1.426968,11320.225,0
285305,2020-09-03 09:33:00.791797,10830.955,-0.489244,-21.341815,0.470896,0.682944,-21.962345,0.562896,11233.745,0.386811,...,11320.005,-22.431528,-24.005183,1.433453,11320.340,-1.570151,-16.401193,1.429919,11320.495,0
285306,2020-09-03 09:33:12.133621,10830.955,-0.921108,-22.820611,0.468831,-1.294354,-7.394954,0.573023,11233.745,0.682944,...,11317.035,-5.331503,-29.535550,1.431562,11320.005,-22.431528,-24.005183,1.433453,11320.340,0
285307,2020-09-03 09:33:23.445957,10830.825,-1.213449,-34.496262,0.496368,0.075363,-25.412812,0.492631,11227.035,-1.294354,...,11317.065,41.016147,-38.257578,1.452187,11317.035,-5.331503,-29.535550,1.431562,11320.005,0


<IPython.core.display.Javascript object>

In [11]:
transformed.to_csv("test_data.csv", index=False)

<IPython.core.display.Javascript object>

In [12]:
test_data = transformed.loc[transformed["time"] > "2020-07-24 07:56:29.780165"]

test_data

,time,midprice,oir,voi,tfi,t-oir-501,t-voi-501,t-tfi-501,t-midprice-501,t-oir-502,...,t-midprice-998,t-oir-999,t-voi-999,t-tfi-999,t-midprice-999,t-oir-1000,t-voi-1000,t-tfi-1000,t-midprice-1000,target
72798,2020-07-24 07:56:41.516815,9537.490,3.465263,1.134861,0.580011,0.662033,86.074191,0.240416,9544.995,1.268136,...,9499.785,1.100954,-30.557809,0.490984,9499.785,1.118696,-32.397356,0.408895,9499.785,0
72799,2020-07-24 07:56:53.097352,9540.295,-9.078244,0.272401,0.544738,0.906631,106.000273,0.224529,9545.725,0.662033,...,9499.875,1.167474,-29.833054,0.385222,9499.785,1.100954,-30.557809,0.490984,9499.785,0
72800,2020-07-24 07:57:04.460598,9540.295,-8.906102,0.275545,0.544732,0.857119,92.169343,0.205053,9546.085,0.906631,...,9499.875,1.197133,-26.758507,0.348068,9499.875,1.167474,-29.833054,0.385222,9499.785,0
72801,2020-07-24 07:57:15.855812,9540.295,-8.852874,0.177546,0.544744,1.235613,76.799881,0.169118,9550.815,0.857119,...,9502.775,1.462656,-14.226084,0.348108,9499.875,1.197133,-26.758507,0.348068,9499.875,0
72802,2020-07-24 07:57:27.213732,9540.295,-4.058831,0.180030,0.544744,0.500081,93.587702,0.197177,9550.815,1.235613,...,9502.775,0.615400,11.463975,0.281089,9502.775,1.462656,-14.226084,0.348108,9499.875,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285304,2020-09-03 09:32:49.476146,10830.955,-0.043333,-22.368282,0.461289,0.386811,-4.001545,0.539549,11237.055,-0.260199,...,11320.340,-1.570151,-16.401193,1.429919,11320.495,-1.441880,-15.786901,1.426968,11320.225,0
285305,2020-09-03 09:33:00.791797,10830.955,-0.489244,-21.341815,0.470896,0.682944,-21.962345,0.562896,11233.745,0.386811,...,11320.005,-22.431528,-24.005183,1.433453,11320.340,-1.570151,-16.401193,1.429919,11320.495,0
285306,2020-09-03 09:33:12.133621,10830.955,-0.921108,-22.820611,0.468831,-1.294354,-7.394954,0.573023,11233.745,0.682944,...,11317.035,-5.331503,-29.535550,1.431562,11320.005,-22.431528,-24.005183,1.433453,11320.340,0
285307,2020-09-03 09:33:23.445957,10830.825,-1.213449,-34.496262,0.496368,0.075363,-25.412812,0.492631,11227.035,-1.294354,...,11317.065,41.016147,-38.257578,1.452187,11317.035,-5.331503,-29.535550,1.431562,11320.005,0


<IPython.core.display.Javascript object>

In [13]:
model = load("BTC_100m_forecast.joblib")

<IPython.core.display.Javascript object>

In [14]:
predictions = model.predict(
    test_data.drop(columns=["time", "midprice", "oir", "voi", "tfi", "target"])
)
y_true = test_data["target"]

<IPython.core.display.Javascript object>

In [15]:
test_data = test_data.copy()
test_data["probs 0"] = model.predict_proba(
    test_data.drop(columns=["time", "midprice", "oir", "voi", "tfi", "target"])
)[0:, 0:1]
test_data["probs 1"] = 1 - test_data["probs 0"]
test_data["prediction"] = predictions

test_data

,time,midprice,oir,voi,tfi,t-oir-501,t-voi-501,t-tfi-501,t-midprice-501,t-oir-502,...,t-tfi-999,t-midprice-999,t-oir-1000,t-voi-1000,t-tfi-1000,t-midprice-1000,target,probs 0,probs 1,prediction
72798,2020-07-24 07:56:41.516815,9537.490,3.465263,1.134861,0.580011,0.662033,86.074191,0.240416,9544.995,1.268136,...,0.490984,9499.785,1.118696,-32.397356,0.408895,9499.785,0,0.460191,0.539809,1
72799,2020-07-24 07:56:53.097352,9540.295,-9.078244,0.272401,0.544738,0.906631,106.000273,0.224529,9545.725,0.662033,...,0.385222,9499.785,1.100954,-30.557809,0.490984,9499.785,0,0.463564,0.536436,1
72800,2020-07-24 07:57:04.460598,9540.295,-8.906102,0.275545,0.544732,0.857119,92.169343,0.205053,9546.085,0.906631,...,0.348068,9499.875,1.167474,-29.833054,0.385222,9499.785,0,0.469798,0.530202,1
72801,2020-07-24 07:57:15.855812,9540.295,-8.852874,0.177546,0.544744,1.235613,76.799881,0.169118,9550.815,0.857119,...,0.348108,9499.875,1.197133,-26.758507,0.348068,9499.875,0,0.477805,0.522195,1
72802,2020-07-24 07:57:27.213732,9540.295,-4.058831,0.180030,0.544744,0.500081,93.587702,0.197177,9550.815,1.235613,...,0.281089,9502.775,1.462656,-14.226084,0.348108,9499.875,0,0.481908,0.518092,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285304,2020-09-03 09:32:49.476146,10830.955,-0.043333,-22.368282,0.461289,0.386811,-4.001545,0.539549,11237.055,-0.260199,...,1.429919,11320.495,-1.441880,-15.786901,1.426968,11320.225,0,0.469778,0.530222,1
285305,2020-09-03 09:33:00.791797,10830.955,-0.489244,-21.341815,0.470896,0.682944,-21.962345,0.562896,11233.745,0.386811,...,1.433453,11320.340,-1.570151,-16.401193,1.429919,11320.495,0,0.467412,0.532588,1
285306,2020-09-03 09:33:12.133621,10830.955,-0.921108,-22.820611,0.468831,-1.294354,-7.394954,0.573023,11233.745,0.682944,...,1.431562,11320.005,-22.431528,-24.005183,1.433453,11320.340,0,0.463614,0.536386,1
285307,2020-09-03 09:33:23.445957,10830.825,-1.213449,-34.496262,0.496368,0.075363,-25.412812,0.492631,11227.035,-1.294354,...,1.452187,11317.035,-5.331503,-29.535550,1.431562,11320.005,0,0.468563,0.531437,1


<IPython.core.display.Javascript object>

In [16]:
test_data["target"] = y_true
test_data = test_data.drop(columns=["oir", "voi", "tfi"])

test_data

,time,midprice,t-oir-501,t-voi-501,t-tfi-501,t-midprice-501,t-oir-502,t-voi-502,t-tfi-502,t-midprice-502,...,t-tfi-999,t-midprice-999,t-oir-1000,t-voi-1000,t-tfi-1000,t-midprice-1000,target,probs 0,probs 1,prediction
72798,2020-07-24 07:56:41.516815,9537.490,0.662033,86.074191,0.240416,9544.995,1.268136,91.206296,0.240392,9544.995,...,0.490984,9499.785,1.118696,-32.397356,0.408895,9499.785,0,0.460191,0.539809,1
72799,2020-07-24 07:56:53.097352,9540.295,0.906631,106.000273,0.224529,9545.725,0.662033,86.074191,0.240416,9544.995,...,0.385222,9499.785,1.100954,-30.557809,0.490984,9499.785,0,0.463564,0.536436,1
72800,2020-07-24 07:57:04.460598,9540.295,0.857119,92.169343,0.205053,9546.085,0.906631,106.000273,0.224529,9545.725,...,0.348068,9499.875,1.167474,-29.833054,0.385222,9499.785,0,0.469798,0.530202,1
72801,2020-07-24 07:57:15.855812,9540.295,1.235613,76.799881,0.169118,9550.815,0.857119,92.169343,0.205053,9546.085,...,0.348108,9499.875,1.197133,-26.758507,0.348068,9499.875,0,0.477805,0.522195,1
72802,2020-07-24 07:57:27.213732,9540.295,0.500081,93.587702,0.197177,9550.815,1.235613,76.799881,0.169118,9550.815,...,0.281089,9502.775,1.462656,-14.226084,0.348108,9499.875,0,0.481908,0.518092,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285304,2020-09-03 09:32:49.476146,10830.955,0.386811,-4.001545,0.539549,11237.055,-0.260199,4.340889,0.405184,11238.525,...,1.429919,11320.495,-1.441880,-15.786901,1.426968,11320.225,0,0.469778,0.530222,1
285305,2020-09-03 09:33:00.791797,10830.955,0.682944,-21.962345,0.562896,11233.745,0.386811,-4.001545,0.539549,11237.055,...,1.433453,11320.340,-1.570151,-16.401193,1.429919,11320.495,0,0.467412,0.532588,1
285306,2020-09-03 09:33:12.133621,10830.955,-1.294354,-7.394954,0.573023,11233.745,0.682944,-21.962345,0.562896,11233.745,...,1.431562,11320.005,-22.431528,-24.005183,1.433453,11320.340,0,0.463614,0.536386,1
285307,2020-09-03 09:33:23.445957,10830.825,0.075363,-25.412812,0.492631,11227.035,-1.294354,-7.394954,0.573023,11233.745,...,1.452187,11317.035,-5.331503,-29.535550,1.431562,11320.005,0,0.468563,0.531437,1


<IPython.core.display.Javascript object>

In [17]:
test_data["trade time"] = test_data["time"].shift(501)

<IPython.core.display.Javascript object>

In [27]:
test_data.to_csv("test_data_predictions.csv", index=False)

<IPython.core.display.Javascript object>

In [58]:
class MarketSim:
    def __init__(self, test_data, confidence, risk, reward, position_size, balance):
        self.test_data = test_data
        self.confidence = confidence
        self.risk = risk
        self.reward = reward
        self.position_size = position_size
        self.balance = balance
        self.starting_balance = balance
        self.get_predictions()

    def get_predictions(self):
        confidence_df = self.test_data[
            (self.test_data["probs 0"] > self.confidence)
            | (self.test_data["probs 1"] > self.confidence)
        ]
        confidence_df["success"] = "no"
        confidence_df.loc[
            confidence_df["prediction"] == confidence_df["target"], "success"
        ] = "yes"

        live_test = confidence_df[
            ["trade time", "t-midprice-501", "midprice", "prediction", "success"]
        ]
        live_test = live_test.rename(
            columns={"trade time": "time", "midprice": "future midprice"}
        )
        live_test = live_test.reset_index(drop=True)

        live_test = live_test.merge(
            data[["time", "midprice"]].reset_index(drop=True), on="time"
        )
        live_test = live_test.append(data[["time", "midprice"]].reset_index(drop=True))
        live_test = live_test.set_index("time")
        live_test = live_test.sort_index()

        self.test_data = live_test

    def test(self):
        position = ""
        trade_price = 0
        self.profit_list = []
        self.open_times = []
        self.close_times = []
        self.open_prices = []
        self.close_prices = []
        self.direction_list = []

        print(self.test_data.info())

        for index, row in self.test_data.iterrows():
            if row["prediction"] == 0 and position == "":
                position = "short"
                trade_price = row["t-midprice-501"]
                #                 + (row["t-midprice-501"] * 0.0001)
                self.open_times.append(index)
                self.open_prices.append(trade_price)
                self.direction_list.append(position)
            elif row["prediction"] == 1 and position == "":
                position = "long"
                trade_price = row["t-midprice-501"]
                #                 - (row["t-midprice-501"] * 0.0001)
                self.open_times.append(index)
                self.open_prices.append(trade_price)
                self.direction_list.append(position)
            if position == "short":
                if row["midprice"] <= trade_price - (trade_price * self.reward):
                    close_price = row["midprice"]
                    profit = (trade_price - close_price) * (
                        self.position_size / trade_price
                    )
                    profit += self.position_size * 0.00025
                    profit += self.position_size * 0.00025
                    profit = round(profit, 2)
                    self.profit_list.append(profit)
                    position = ""
                    self.close_times.append(index)
                    self.close_prices.append(row["midprice"])
                    self.balance += profit
                elif row["midprice"] >= trade_price + (trade_price * self.risk):
                    close_price = row["midprice"]
                    profit = (trade_price - close_price) * (
                        self.position_size / trade_price
                    )
                    profit += self.position_size * 0.00025
                    profit += self.position_size * 0.00025
                    profit = round(profit, 2)
                    self.profit_list.append(profit)
                    position = ""
                    self.close_times.append(index)
                    self.close_prices.append(row["midprice"])
                    self.balance += profit
            if position == "long":
                if row["midprice"] >= trade_price + (trade_price * self.reward):
                    close_price = row["midprice"]
                    profit = (
                        (trade_price - close_price)
                        * (self.position_size / trade_price)
                        * -1
                    )
                    profit += self.position_size * 0.00025
                    profit += self.position_size * 0.00025
                    profit = round(profit, 2)
                    self.profit_list.append(profit)
                    position = ""
                    self.close_times.append(index)
                    self.close_prices.append(row["midprice"])
                    self.balance += profit
                elif row["midprice"] <= trade_price - (trade_price * self.risk):
                    close_price = row["midprice"]
                    profit = (
                        (trade_price - close_price)
                        * (self.position_size / trade_price)
                        * -1
                    )
                    profit += self.position_size * 0.00025
                    profit += self.position_size * 0.00025
                    profit = round(profit, 2)
                    self.profit_list.append(profit)
                    position = ""
                    self.close_times.append(index)
                    self.close_prices.append(row["midprice"])
                    self.balance += profit

            self.position_size = self.balance

        self.equity_list = []
        for i in self.profit_list:
            self.equity_list.append(i + self.starting_balance)
            self.starting_balance = i + self.starting_balance

    def get_trades(self):
        trades_df = pd.DataFrame(self.open_times, columns=["open time"])
        trades_df["key"] = trades_df.index
        trades_df["open price"] = self.open_prices

        try:
            trades_df["close time"] = self.close_times
            trades_df["close price"] = self.close_prices
            trades_df["equity"] = self.equity_list
            trades_df["PnL"] = self.profit_list
        except (ValueError):
            self.close_times.append(np.nan)
            self.close_prices.append(np.nan)
            self.profit_list.append(np.nan)
            self.equity_list.append(np.nan)
            trades_df["close time"] = self.close_times
            trades_df["close price"] = self.close_prices
            trades_df["equity"] = self.equity_list
            trades_df["PnL"] = self.profit_list

        trades_df["position"] = self.direction_list

        trades_close = trades_df.drop(columns=["open time", "open price"])
        trades_open = trades_df.drop(columns=["close time", "close price"])
        trades_close["OC"] = "close"
        trades_open["OC"] = "open"

        trades_open = trades_open.rename(
            columns={"open time": "time", "open price": "price"}
        )
        trades_close = trades_close.rename(
            columns={"close time": "time", "close price": "price"}
        )

        trades_df = trades_open.append(trades_close)
        trades_df["key"] = trades_df["key"].astype(str)

        self.trades_df = trades_df

        return trades_df

    def display_trades(self):
        fig = px.scatter(
            self.trades_df,
            x="time",
            y="price",
            color="key",
            hover_data=["OC", "position", "PnL"],
        )
        fig.show()

    def display_equity(self):
        equity_df = self.trades_df.copy().drop_duplicates(subset=["key"])
        fig = px.line(equity_df, x="time", y="equity")
        fig.show()

<IPython.core.display.Javascript object>

In [28]:
test_data = pd.read_csv("test_data_predictions.csv")

<IPython.core.display.Javascript object>

In [31]:
test_data = test_data.dropna()
test_data

,time,midprice,t-oir-501,t-voi-501,t-tfi-501,t-midprice-501,t-oir-502,t-voi-502,t-tfi-502,t-midprice-502,...,t-midprice-999,t-oir-1000,t-voi-1000,t-tfi-1000,t-midprice-1000,target,probs 0,probs 1,prediction,trade time
501,2020-07-24 13:23:04.294420,9553.225,1.249740,-22.935431,0.345302,9571.100,0.618170,-9.812822,0.255617,9571.565,...,9520.005,0.836111,-40.515293,0.886006,9520.005,0,0.509787,0.490213,0,2020-07-24 07:56:41.516815
502,2020-07-24 13:23:15.568196,9553.895,-1.363672,-64.326887,0.355308,9570.005,1.249740,-22.935431,0.345302,9571.100,...,9520.005,0.322191,-48.169109,0.902386,9520.005,0,0.517400,0.482600,0,2020-07-24 07:56:53.097352
503,2020-07-24 13:23:26.768205,9554.285,4.017513,-42.612788,0.341127,9568.595,-1.363672,-64.326887,0.355308,9570.005,...,9520.005,0.079639,-50.363275,0.901905,9520.005,0,0.521668,0.478332,0,2020-07-24 07:57:04.460598
504,2020-07-24 13:23:38.163816,9554.285,1.185701,-41.038011,0.353265,9568.595,4.017513,-42.612788,0.341127,9568.595,...,9520.005,0.954171,-54.771372,0.821617,9520.005,0,0.529201,0.470799,0,2020-07-24 07:57:15.855812
505,2020-07-24 13:23:49.464790,9554.285,1.759679,-39.352717,0.258099,9568.595,1.185701,-41.038011,0.353265,9568.595,...,9520.005,0.965269,-53.571857,0.900828,9520.005,0,0.530649,0.469351,0,2020-07-24 07:57:27.213732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209018,2020-09-03 09:32:49.476146,10830.955,0.386811,-4.001545,0.539549,11237.055,-0.260199,4.340889,0.405184,11238.525,...,11320.495,-1.441880,-15.786901,1.426968,11320.225,0,0.469778,0.530222,1,2020-09-03 07:56:39.068704
209019,2020-09-03 09:33:00.791797,10830.955,0.682944,-21.962345,0.562896,11233.745,0.386811,-4.001545,0.539549,11237.055,...,11320.340,-1.570151,-16.401193,1.429919,11320.495,0,0.467412,0.532588,1,2020-09-03 07:56:50.473710
209020,2020-09-03 09:33:12.133621,10830.955,-1.294354,-7.394954,0.573023,11233.745,0.682944,-21.962345,0.562896,11233.745,...,11320.005,-22.431528,-24.005183,1.433453,11320.340,0,0.463614,0.536386,1,2020-09-03 07:57:01.809474
209021,2020-09-03 09:33:23.445957,10830.825,0.075363,-25.412812,0.492631,11227.035,-1.294354,-7.394954,0.573023,11233.745,...,11317.035,-5.331503,-29.535550,1.431562,11320.005,0,0.468563,0.531437,1,2020-09-03 07:57:13.360853


<IPython.core.display.Javascript object>

In [34]:
test_data[["time", "trade time"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208522 entries, 501 to 209022
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time        208522 non-null  object
 1   trade time  208522 non-null  object
dtypes: object(2)
memory usage: 4.8+ MB


<IPython.core.display.Javascript object>

In [37]:
for col in ["time", "trade time"]:
    test_data[col] = pd.to_datetime(test_data[col])

test_data

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,time,midprice,t-oir-501,t-voi-501,t-tfi-501,t-midprice-501,t-oir-502,t-voi-502,t-tfi-502,t-midprice-502,...,t-midprice-999,t-oir-1000,t-voi-1000,t-tfi-1000,t-midprice-1000,target,probs 0,probs 1,prediction,trade time
501,2020-07-24 13:23:04.294420,9553.225,1.249740,-22.935431,0.345302,9571.100,0.618170,-9.812822,0.255617,9571.565,...,9520.005,0.836111,-40.515293,0.886006,9520.005,0,0.509787,0.490213,0,2020-07-24 07:56:41.516815
502,2020-07-24 13:23:15.568196,9553.895,-1.363672,-64.326887,0.355308,9570.005,1.249740,-22.935431,0.345302,9571.100,...,9520.005,0.322191,-48.169109,0.902386,9520.005,0,0.517400,0.482600,0,2020-07-24 07:56:53.097352
503,2020-07-24 13:23:26.768205,9554.285,4.017513,-42.612788,0.341127,9568.595,-1.363672,-64.326887,0.355308,9570.005,...,9520.005,0.079639,-50.363275,0.901905,9520.005,0,0.521668,0.478332,0,2020-07-24 07:57:04.460598
504,2020-07-24 13:23:38.163816,9554.285,1.185701,-41.038011,0.353265,9568.595,4.017513,-42.612788,0.341127,9568.595,...,9520.005,0.954171,-54.771372,0.821617,9520.005,0,0.529201,0.470799,0,2020-07-24 07:57:15.855812
505,2020-07-24 13:23:49.464790,9554.285,1.759679,-39.352717,0.258099,9568.595,1.185701,-41.038011,0.353265,9568.595,...,9520.005,0.965269,-53.571857,0.900828,9520.005,0,0.530649,0.469351,0,2020-07-24 07:57:27.213732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209018,2020-09-03 09:32:49.476146,10830.955,0.386811,-4.001545,0.539549,11237.055,-0.260199,4.340889,0.405184,11238.525,...,11320.495,-1.441880,-15.786901,1.426968,11320.225,0,0.469778,0.530222,1,2020-09-03 07:56:39.068704
209019,2020-09-03 09:33:00.791797,10830.955,0.682944,-21.962345,0.562896,11233.745,0.386811,-4.001545,0.539549,11237.055,...,11320.340,-1.570151,-16.401193,1.429919,11320.495,0,0.467412,0.532588,1,2020-09-03 07:56:50.473710
209020,2020-09-03 09:33:12.133621,10830.955,-1.294354,-7.394954,0.573023,11233.745,0.682944,-21.962345,0.562896,11233.745,...,11320.005,-22.431528,-24.005183,1.433453,11320.340,0,0.463614,0.536386,1,2020-09-03 07:57:01.809474
209021,2020-09-03 09:33:23.445957,10830.825,0.075363,-25.412812,0.492631,11227.035,-1.294354,-7.394954,0.573023,11233.745,...,11317.035,-5.331503,-29.535550,1.431562,11320.005,0,0.468563,0.531437,1,2020-09-03 07:57:13.360853


<IPython.core.display.Javascript object>

In [85]:
test = MarketSim(test_data, 0.60, 0.01, 0.02, 10000, 10000)

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<IPython.core.display.Javascript object>

In [86]:
test.test()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 348168 entries, 2020-07-02 13:57:49.545534 to 2020-09-03 09:33:34.843553
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   t-midprice-501   62859 non-null   float64
 1   future midprice  62859 non-null   float64
 2   prediction       62859 non-null   float64
 3   success          62859 non-null   object 
 4   midprice         348168 non-null  float64
dtypes: float64(4), object(1)
memory usage: 15.9+ MB
None


<IPython.core.display.Javascript object>

In [87]:
test.balance

12330.720000000005

<IPython.core.display.Javascript object>

In [90]:
test.get_trades()

,time,key,price,equity,PnL,position,OC
0,2020-07-24 08:06:56.357093,0,9559.735,9867.52,-132.48,short,open
1,2020-07-25 22:51:33.678075,1,9692.915,9772.31,-95.21,short,open
2,2020-07-26 04:59:27.335955,2,9787.745,9678.10,-94.21,short,open
3,2020-07-26 06:02:57.186353,3,9896.545,9583.73,-94.37,short,open
4,2020-07-26 06:03:53.465342,4,9997.995,9490.75,-92.98,short,open
...,...,...,...,...,...,...,...
99,2020-09-02 07:03:42.394239,99,11519.575,12099.30,-117.89,long,close
100,2020-09-02 07:10:08.873714,100,11351.055,11959.30,-140.00,long,close
101,2020-09-02 07:22:43.377274,101,11170.080,11833.99,-125.31,long,close
102,2020-09-02 07:49:14.987735,102,11398.545,12081.95,247.96,long,close


<IPython.core.display.Javascript object>

In [91]:
test.display_trades()

<IPython.core.display.Javascript object>

In [92]:
test.display_equity()

<IPython.core.display.Javascript object>